# Language Services

Azure Cognitive Services includes a pair of services that are backed by deep-learning models trained to perform natural language processing (NLP). The [Language](https://azure.microsoft.com/en-us/services/cognitive-services/language-service/#overview) service provides APIs for sentiment analysis, named-entity recognition, question answering, key-phrase extraction, language understanding, and more. Among its many uses is building chatbots that respond intelligently to queries regarding a company’s product or service offerings. There’s also a [Translator](https://azure.microsoft.com/en-us/services/cognitive-services/translator/) service that translates text between more than 100 languages and dialects. Volkswagen uses it to [translate onscreen instructions in cars](https://customers.microsoft.com/en-us/story/779468-volkswagen-azure-automotive-en) to match the language in the owner’s manual and to ensure the quality of the documentation that they produce in more than 40 languages.

The [`TextAnalyticsClient`](https://docs.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python) class in the [Python text-analytics SDK](https://pypi.org/project/azure-ai-textanalytics/) provides a convenient interface to many of the Language service’s features. Here’s an example that analyzes a text sample for sentiment. Replace `KEY` with a subscription key for the Language service and `ENDPOINT` with an endpoint:

In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

client = TextAnalyticsClient(ENDPOINT, AzureKeyCredential(KEY))
input = [{ 'id': '1000', 'text': 'Programming is fun, but the hours are long' }]
response = client.analyze_sentiment(input)

for result in response:
    print(result.confidence_scores)

{'positive': 0.94, 'neutral': 0.05, 'negative': 0.01}


Here’s another example that applies sentiment analysis to multiple text samples with one call to the Language service:

In [2]:
client = TextAnalyticsClient(ENDPOINT, AzureKeyCredential(KEY))

input = [
    { 'id': '1000', 'text': 'Programming is fun, but the hours are long' },
    { 'id': '1001', 'text': 'Great food and excellent service' },
    { 'id': '1002', 'text': 'The product worked as advertised but is overpriced' },
    { 'id': '1003', 'text': 'Moving to the cloud was the best decision we ever made' },
    { 'id': '1004', 'text': 'Programming is so fun I\'d do it for free. Don\'t tell my boss!' }
]

response = client.analyze_sentiment(input)

for result in response:
    text = ''.join([x.text for x in result.sentences])
    print(f'{text} => {result.confidence_scores.positive}')


Programming is fun, but the hours are long => 0.94
Great food and excellent service => 1.0
The product worked as advertised but is overpriced => 0.0
Moving to the cloud was the best decision we ever made => 1.0
Programming is so fun I'd do it for free. Don't tell my boss! => 1.0


Sentiment analysis is one of several operations supported by the `TextAnalyticsClient` class. Another is named-entity recognition. Suppose you’re building a system that sorts and prioritizes support tickets received by your company’s help desk. The [`recognize_entities`](https://docs.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-recognize-entities) method recognizes entities such as people, places, organizations, dates and times, and quantities in input text. It extracts them and reveals the entity types as well:

In [3]:
documents = [
    'My printer isn\'t working. Can someone from IT come to my office and have a look?'
]

results = client.recognize_entities(documents)

for result in results:
    for entity in result.entities:
        print(f'{entity.text} ({entity.category})')

printer (Product)
IT (Skill)
office (Location)


A related method named [`recognize_pii_entities`](https://docs.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-recognize-pii-entities) extracts personally identifiable information (PII) entities such as bank-account info, social-security numbers, and credit-card numbers from input text, while the [`extract_key_phrases`](https://docs.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-extract-key-phrases) method extracts key phrases:

In [4]:
documents = [
    'Natural Language Processing, or NLP, encompasses a variety of activities ' \
    'including text classification, keyword extraction, named-entity recognition, ' \
    'question answering, and language translation.'
]

results = client.extract_key_phrases(documents)

for result in results:
    for phrase in result.key_phrases:
        print(phrase)

Natural Language Processing
language translation
text classification
keyword extraction
question answering
NLP
variety
activities
recognition


`TextAnalyticsClient` provides a wrapper around text-analytics APIs. Other Python SDKs unlock additional features of the Language service. For example, the [question-answering SDK](https://pypi.org/project/azure-ai-language-questionanswering/) provides APIs for answering questions from manuals, FAQs, blog posts, and other documents that you provide. Here's an example that uses the SDK's `QuestionAnsweringClient` class to answer a question from a provided context:

In [5]:
from azure.ai.language.questionanswering import models as qna
from azure.ai.language.questionanswering import QuestionAnsweringClient

client = QuestionAnsweringClient(ENDPOINT, AzureKeyCredential(KEY))

question = 'What is the minimum age required to serve as a United State Senator?'

context = 'No Person shall be a Senator who shall not have attained to the Age of ' \
          'thirty Years, and been nine Years a Citizen of the United States, and ' \
          'who shall not, when elected, be an Inhabitant of that State for which ' \
          'he shall be chosen.'

input = qna.AnswersFromTextOptions(question=question, text_documents=[context])
results = client.get_answers_from_text(input)

answer = results.answers[0].short_answer
print(f'{answer.text} ({answer.confidence:.1%})')

thirty Years (88.0%)


For more information, refer to the article entitled [Azure Cognitive Language Services Question Answering client library for Python](https://docs.microsoft.com/en-us/python/api/overview/azure/ai-language-questionanswering-readme?view=azure-python).

The Translator service uses state-of-the-art neural machine translation to translate text between dozens of languages. It can also identify written languages. Suppose your objective is to translate into English questions written in other languages and submitted through your company’s Web site. First you need to determine whether the source language is English. If it’s not, you want to translate it so you can respond to the customer’s request.

The code below analyzes a text sample and shows the language it’s written in. Microsoft doesn’t currently offer a Python SDK for the Translator service, but you can use Python’s [`requests`](https://pypi.org/project/requests/) package to simplify calls. To demonstrate, create a Translator resource in the Azure Portal and grab the subscription key (the one labeled “Text Translation”), endpoint, and region. Then run the following code, replacing `KEY` and `ENDPOINT` with the key and endpoint and `REGION` with the Azure region you selected:

In [6]:
import requests

input = [{ 'text': 'Quand votre nouveau livre sera-t-il disponible?' }]

headers = {
    'Ocp-Apim-Subscription-Key': KEY,
    'Ocp-Apim-Subscription-Region': REGION,
    'Content-type': 'application/json'
}

uri = ENDPOINT + 'detect?api-version=3.0&to=en'
response = requests.post(uri, headers=headers, json=input)
results = response.json()

print(results[0]['language'])

fr


Now that you’ve determined the source language isn’t English, you can translate it this way:

In [7]:
uri = ENDPOINT + 'translate?api-version=3.0&from=fr&to=en'
response = requests.post(uri, headers=headers, json=input)
results = response.json()

print(results[0]['translations'][0]['text'])

When will your new book be available?


When you obtained an endpoint for the Translator service, did you notice that the Azure Portal offered two endpoints? One for “Text Translation” and another for “Document Translation?” That’s because the Translator service features a second API for translating entire documents, including PDFs. There’s even a [Python SDK](https://pypi.org/project/azure-ai-translation-document/) to help out. The only catch is that documents must first be uploaded to Azure blob storage. For examples showing document-translation API in action, see [Azure Document Translation client library for Python](https://docs.microsoft.com/en-us/python/api/overview/azure/ai-translation-document-readme?view=azure-python). The API is asynchronous and can process batches of documents in one call, so it’s ideal not just for translating individual documents, but for translating large volumes of documents at scale.